In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods, linkage_vis
from elastic_rods import EnergyType, InterleavingType
from bending_validation import suppress_stdout as so
import linkage_optimization

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False
rw = 1
sw = 10
drw = 0.01
dsw = 0.01

In [ ]:
model_path = '../../examples/florin/20181227_193550_meshID_5ca2f7ab-3602-4ede-ac4d-c2bd798b2961.obj'

### Flat Linkage

In [ ]:
flatLinkage = elastic_rods.RodLinkage(model_path, 8, True, InterleavingType.xshell)
driver=flatLinkage.centralJoint()
flatLinkage.setMaterial(elastic_rods.RodMaterial('rectangle', 20000, 0.3, [10, 7]))

with so(): elastic_rods.restlen_solve(flatLinkage)
jdo = flatLinkage.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with so(): elastic_rods.compute_equilibrium(flatLinkage, fixedVars=fixedVars)
flatView = linkage_vis.LinkageViewer(flatLinkage, width=1024, height=640)
flatView.setCameraParams(((2.3969, -1.1451, -0.5262), (0.0765, 0.0848, -0.9935), (0.1111, 0.1865, 0.5316)))
flatView.show()

In [ ]:
flatLinkage.set_design_parameter_config(True, False);

In [ ]:
flatView.averagedMaterialFrames = True 
flatView.averagedCrossSections = True

In [ ]:
flatView.update(scalarField=flatLinkage.stretchingStresses())

### Design Parameter Solve

### Deployed Linkage

In [ ]:
deployedLinkage = elastic_rods.RodLinkage(flatLinkage)
deployedView = linkage_vis.LinkageViewer(deployedLinkage, width=1024, height=640)
deployedView.setCameraParams(((2.3969, -1.1451, -0.5262), (0.0765, 0.0848, -0.9935), (0.1111, 0.1865, 0.5316)))
deployedView.show()

In [ ]:
deployedLinkage.set_design_parameter_config(True, False);

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, deployedLinkage, opts, fv):
    opts.gradTol = 1e-4
    return elastic_rods.compute_equilibrium(deployedLinkage, tgtAngle, options=opts, fixedVars=fv)

In [ ]:
import benchmark
import time
benchmark.reset()
startTime = time.time()

In [ ]:
with so(): open_linkage(deployedLinkage, driver, np.deg2rad(93) - deployedLinkage.averageJointAngle, 40, deployedView, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, useTargetAngleConstraint=True);

In [ ]:
benchmark.report()
duration = time.time() - startTime
print(duration)

In [ ]:
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1
OPTS.beta = 1e-8
OPTS.niter = 25
OPTS.verboseNonPosDef = False

useCenterline = True
optimizer = linkage_optimization.XShellOptimization(flatLinkage, deployedLinkage, OPTS, np.pi / 128)


# optimizer.rl_regularization_weight = rest_length_weight
# optimizer.smoothing_weight = smoothing_weight
# optimizer.beta = 500000.0
# optimizer.gamma = 1

In [ ]:
def callback():
    flatView.update()
    deployedView.update()

In [ ]:
algorithm = linkage_optimization.OptAlgorithm.NEWTON_CG
optimizer.XShellOptimize(algorithm, 2000, 1.0, 1e-2, callback, -1, True, True)